In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# import module we'll need to import our custom module
from shutil import copyfile
import distutils
from distutils import dir_util

distutils.dir_util.copy_tree("../input/proteinproject/project", "../working")

print(os.listdir("../working"))


In [ ]:
print("Working:", os.listdir("../working"))
print("Saved config: ", os.listdir("../input/protein/saved/Protein_Resnet34/0315_232012"))
print("Input:", os.listdir("../input/human-protein-atlas-image-classification"))

In [ ]:
import os
import argparse
import torch
from tqdm import tqdm
import data_loader.data_loaders as module_data
import model.resnet as module_arch

def get_instance(module, name, config, *args):
    return getattr(module, config[name]['type'])(*args, **config[name]['args'])

def test(resume):
    checkpoint = torch.load(resume)
    config = checkpoint['config']
    
    # setup data_loader instances
    data_loader = getattr(module_data, config['data_loader']['type'])(
        config['data_loader']['args']['data_dir'],
        batch_size=1,
        shuffle=False,
        validation_split=0.0,
        training=False,
        num_workers=0
    )

    # build model architecture
    model = get_instance(module_arch, 'arch', config)
    model.summary()

    # load state dict
    state_dict = checkpoint['state_dict']
    if config['n_gpu'] > 1:
        model = torch.nn.DataParallel(model)
    model.load_state_dict(state_dict)

    # prepare model for testing
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()

    sample_submission = pd.read_csv("../input/human-protein-atlas-image-classification/sample_submission.csv")
    
    filenames, labels, submissions = [], [], []
    with torch.no_grad():
        for i, (data, target) in enumerate(tqdm(data_loader)):
            data = data.to(device)
            output = model(data)
            label = output.sigmoid().cpu().data.numpy()
            
            filenames.append(target)
            labels.append(label > 0.15)
            
    for row in np.concatenate(labels):
        subrow = ' '.join(list([str(i) for i in np.nonzero(row)[0]]))
        submissions.append(subrow)
    sample_submission['Predicted'] = submissions
    os.mkdir("./submit")
    sample_submission.to_csv('./submit/submission.csv', index=None)
            
            
# Run!
test("../input/protein/saved/Protein_Resnet34/0315_232012/checkpoint-epoch10.pth")

In [ ]:
print("End")